In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, LSTM, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import class_weight
# from imblearn.over_sampling import SMOTE
import argparse
from joblib import dump
import matplotlib.pyplot as plt


# parser = argparse.ArgumentParser()

# parser.add_argument('directory', 
#                     type=str, 
#                     help = 'directory where files are stored')

# parser.add_argument('file_name',
#                     type = str,
#                     help = 'dos traffic file')


# args = parser.parse_args()


base_dir = '../data/Car Hacking Dataset/'
data_path = os.path.join(base_dir, 'preprocessed_car_hacking.csv')
data = pd.read_csv(data_path)

data.drop(['Timestamp'], axis = 1, inplace=True)

data = data[:100_000]

2023-10-30 15:58:07.735338: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
def hex_to_bin(hex_num):
    
    binary_value = bin(int(str(hex_num), 16))[2:]
    
    return binary_value

def int_to_bin(int_num):
    
    binary_value = bin(int_num)[2:]
    
    return binary_value

def pad(value, length):
    
    curr_length = len(str(value))
    
    zeros = '0' * (length - curr_length)
    
    return zeros + value

hex_to_dec = lambda x: int(x, 16)

def transform_data(data):

    data['ID'] = data['ID'].apply(hex_to_dec)
    data['Payload'] = data['Payload'].apply(hex_to_dec)

    return data

In [16]:
def sequencify_data(X, y, seq_size=10):
    max_index = len(X) - seq_size + 1

    X_seq = []
    y_seq = []

    for i in range(0, max_index, seq_size):
        X_seq.append(X[i:i+seq_size])  # Append the sequence from DataFrame 'X'
        y_seq.append(1 if 1 in y[i:i+seq_size] else 0)  # Check for '1' in 'y' values

    return np.array(X_seq), np.array(y_seq)

# data = transform_data(data)

# X = data.drop('label', axis = 1)
# y = data['label']

In [5]:
print(X.shape)

(100000, 4)


In [17]:
X_seq, y_seq = sequencify_data(X.values, y.values)

In [12]:
X.shape

(100000, 4)

In [13]:
X.iloc[0]

ID         7.900000e+02
DLC        8.000000e+00
Payload    3.696910e+17
IAT        0.000000e+00
Name: 0, dtype: float64

In [14]:
X

,ID,DLC,Payload,IAT
0,790,8,369691032840896623,0.000000
1,399,8,18328243108518297600,0.000209
2,608,8,1810766114876976442,0.000228
3,672,8,7205928855671061760,0.000232
4,809,8,4664461563235663892,0.000237
...,...,...,...,...
99995,305,8,17473685079766010365,0.000213
99996,320,8,136653152,0.000241
99997,1088,8,18374686483960826112,0.000241
99998,399,8,18326272783681650688,0.000244
